# Task 4 - Naive Bayes Classifier 

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

## Import data

In [3]:
main_df = pd.read_csv('data/winequality-red.csv', sep=';')
main_df

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0               7.4             0.700         0.00             1.9      0.076   
1               7.8             0.880         0.00             2.6      0.098   
2               7.8             0.760         0.04             2.3      0.092   
3              11.2             0.280         0.56             1.9      0.075   
4               7.4             0.700         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
1594            6.2             0.600         0.08             2.0      0.090   
1595            5.9             0.550         0.10             2.2      0.062   
1596            6.3             0.510         0.13             2.3      0.076   
1597            5.9             0.645         0.12             2.0      0.075   
1598            6.0             0.310         0.47             3.6      0.067   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                    11.0                  34.0  0.99780  3.51       0.56   
1                    25.0                  67.0  0.99680  3.20       0.68   
2                    15.0                  54.0  0.99700  3.26       0.65   
3                    17.0                  60.0  0.99800  3.16       0.58   
4                    11.0                  34.0  0.99780  3.51       0.56   
...                   ...                   ...      ...   ...        ...   
1594                 32.0                  44.0  0.99490  3.45       0.58   
1595                 39.0                  51.0  0.99512  3.52       0.76   
1596                 29.0                  40.0  0.99574  3.42       0.75   
1597                 32.0                  44.0  0.99547  3.57       0.71   
1598                 18.0                  42.0  0.99549  3.39       0.66   

      alcohol  quality  
0         9.4        5  
1         9.8        5  
2         9.8        5  
3         9.8        6  
4         9.4        5  
...       ...      ...  
1594     10.5        5  
1595     11.2        6  
1596     11.0        6  
1597     10.2        5  
1598     11.0        6  

[1599 rows x 12 columns]

## Validation methods
#### 1) K-fold cross validation
k-fold cross validation split

In [4]:
def cross_validation_split(df, folds=5):
    return np.array_split(df, folds)

#### 2) Train and test datasets

In [29]:
def get_train_and_test_datasets(df, train_to_test_ratio):
    train = df.sample(frac=train_to_test_ratio)
    test = df.drop(train.index)
    return train, test

### Class probabilities
Probabilities are calculated separatelty for each class.

This function determines probabilities that given row belongs to a class.

In [6]:
def get_class_probabilities(train, row):
    by_class = train.groupby('quality')
    prior = by_class.size().div(len(train))
    probabilities = dict()
    classes = by_class.groups.keys()
    total_rows = main_df.shape[0]
    for class_value in classes:

        mean = by_class.get_group(class_value).drop(columns=['quality']).mean()
        std = by_class.get_group(class_value).drop(columns=['quality']).std()
        probabilities[class_value] = prior[class_value]
        
        for prob in norm(mean, std).pdf(row):
            probabilities[class_value] *= prob
            
    return probabilities

#### Return the class with the highest probability

In [7]:
def get_prediction(probabilities):
    return max(probabilities, key=probabilities.get)

#### Accuracy of the model

In [8]:
def get_accuracy(train, test):
    good = 0
    for num in range(test.shape[0]):
        real_class = test.iloc[num]["quality"]
        row = test.drop(columns="quality").iloc[num]
        probabs = get_class_probabilities(train, row)
        pred = get_prediction(probabs)
        if pred ==real_class:
            good+=1
    return good/test.shape[0]

In [9]:
def get_accuracy_cross_validation(df, folds=5):
    accuracy_array = []
    dataset_split = cross_validation_split(df, folds)
    for index, dataset in enumerate(dataset_split):
        train_datasets = np.delete(dataset_split, index)
        train = pd.concat(train_datasets)
        test = dataset
        print("Split ", index+1, ": ", get_accuracy(train, test))
        accuracy_array.append(get_accuracy(train, test))
    average_accuracy = sum(accuracy_array)/len(accuracy_array)
    print("\nAverage accuracy: ", average_accuracy)
    return average_accuracy

## Model validation

### 1) k fold validation ( k = 5 )

##### Experiment 1

In [385]:
get_accuracy_cross_validation(main_df, 5);

Split  1 :  0.521875
Split  2 :  0.490625
Split  3 :  0.540625
Split  4 :  0.4875
Split  5 :  0.5862068965517241

Average accuracy:  0.5253663793103448


##### Note:
There's no need to make more experiments - results would be the same. Function `cross_validation_split(df, folds)` returns the same split every time.

### 2) Train and test datasets (60/40)
Every time `get_train_and_test_datasets(df, folds)` is runned, it generates different datasets. It randomly choses rows to be in the train and test datasets

##### Experiment 1

In [31]:
train, test = get_train_and_test_datasets(main_df, 0.6)
get_accuracy(train, test)

0.4609375

##### Experiment 2

In [32]:
train, test = get_train_and_test_datasets(main_df, 0.6)
get_accuracy(train, test)

0.5640625

##### Experiment 3

In [36]:
train, test = get_train_and_test_datasets(main_df, 0.6)
get_accuracy(train, test)

0.5828125